In [1]:
from pyamaze import maze,COLOR,agent
import numpy as np
import time

In [2]:
col_value=20
row_vaule=20
test_maze=maze(rows=row_vaule,cols=col_value)
test_maze.CreateMaze(loopPercent=10)
# test_maze.CreateMaze(loadMaze='maze--mul_20_20.csv')
test_maze_mapping=test_maze.maze_map

In [3]:
# Retruns a 2d Array With 1 representing path and 
def get_Matrix_mapping(test_maze):
    key_map=dict()
    row=test_maze.rows
    col=test_maze.cols
    maze_mapping = test_maze.maze_map
    maze_2d=np.full(((2*(row))-1,(2*(col))-1),-1)
    maze_2d=np.pad(maze_2d, 1, mode='constant',constant_values= 1)
    row_val_matrix=maze_2d.shape[0]
    col_val_matrix=maze_2d.shape[1]
    ## Filling Boarders with Walls 
    for r in range(row_val_matrix):
        if(r % 2 !=0):
            continue
        for c in range(col_val_matrix):
                maze_2d[r][c]=1
    for c in range(col_val_matrix):
        if(c % 2 !=0):
            continue
        for r in range(row_val_matrix):
            maze_2d[r][c]=1
    for row_map in range(row):
        old_row=row_map+1
        new_row=old_row+row_map
        for col_map in range (col):
            old_col=col_map+1
            new_col=old_col+col_map
            maze_2d[new_row][new_col]=0
            new_points=(new_row,new_col)
            old_points=(old_row,old_col)
            key_map[new_points]=old_points
            if(maze_mapping[(old_row,old_col)]["W"]==1):
                maze_2d[new_row][(new_col-1)]=0
            else:
                maze_2d[new_row][(new_col-1)]=1 
            if(maze_mapping[(old_row,old_col)]["E"]==1):
                maze_2d[new_row][(new_col+1)]=0
            else:
                maze_2d[new_row][(new_col+1)]=1 
            if(maze_mapping[(old_row,old_col)]["N"]==1):
                maze_2d[(new_row-1)][(new_col)]=0
            else:
                maze_2d[(new_row-1)][(new_col)]=1 
            if(maze_mapping[(old_row,old_col)]["S"]==1):
                maze_2d[(new_row+1)][(new_col)]=0
            else:
                maze_2d[(new_row+1)][(new_col)]=1
    return maze_2d,key_map

    

In [4]:
def generate_reward_function(test_maze_2d,row,col):
    reward_matrix = np.full(test_maze_2d.shape, 0.0)
    reward_matrix[test_maze_2d == 1] = -1.0
    reward_matrix[test_maze_2d == 0] = -0.1
    #Starting Cell 
    reward_matrix[1, 1] = -100.0
    #Goal Cell
    reward_matrix[(2*(row))-1,(2*(col))-1] = 10.0
    return reward_matrix

In [5]:
def mdp_value_iteration(test_maze_2d,row,col,reward_matrix,discount_factor=0.9,tolerance=0.0001):
    value_function = np.zeros_like(test_maze_2d, dtype=np.float32)
    actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    max_iterations = 1000
    no_iteration=0
    # Perform value iteration
    for i in range(max_iterations):
        no_iteration=no_iteration+1
        delta = 0
        for x in range(test_maze_2d.shape[0]):
            for y in range(test_maze_2d.shape[1]):
                if test_maze_2d[x, y] == 1:
                    continue
                old_value = value_function[x, y]
                new_value = -np.inf
                for action in actions:
                    new_x = x + action[0]
                    new_y = y + action[1]
                    if 0 <= new_x < test_maze_2d.shape[0] and 0 <= new_y < test_maze_2d.shape[1] and test_maze_2d[new_x, new_y] == 0:
                        v = reward_matrix[new_x, new_y] + discount_factor * value_function[new_x, new_y]
                        if v > new_value:
                            new_value = v
                value_function[x, y] = new_value
                delta = max(delta, np.abs(old_value - new_value))
        if delta < tolerance:
            print("Done")
            print("no_iteration",no_iteration)
            break
        # Find the optimal path from start to goal
    path = [(1, 1)]
    x, y = 1, 1
    counter=0
    while (x, y) != (row, col):
        counter= counter+1
        best_value = -np.inf
        best_action = None
        for action in actions:
            new_x = x + action[0]
            new_y = y + action[1]
            if 0 <= new_x < test_maze_2d.shape[0] and 0 <= new_y < test_maze_2d.shape[1] and test_maze_2d[new_x, new_y] == 0:
                v = reward_matrix[new_x, new_y] + discount_factor * value_function[new_x, new_y]
                if v > best_value:
                    best_value = v
                    best_action = action
        x += best_action[0]
        y += best_action[1]
        path.append((x, y))

    # Print the optimal path
    print("Optimal path:", path)
    print("counter",counter)
            
    return path,counter,no_iteration
    

In [6]:
#val=mdp_value_iteration(test_maze_2d,test_maze.rows,test_maze.cols,reward_matrix,discount_factor=0.9, tolerance=0.0001)

In [7]:
def map_optimal_path_maze(optimal_path,maze_matrix_key_mapping):
    maze_path_val=list()
    for ele in optimal_path:
        if(ele in maze_matrix_key_mapping):
            maze_path_val.append(k[ele])
    final_path=dict()
    for i in range(1,len(maze_path_val)):
        final_path[maze_path_val[i]]=maze_path_val[i-1]
    return final_path
    
    
    

In [8]:
test_maze_2d, k=get_Matrix_mapping(test_maze)
reward_matrix = generate_reward_function(test_maze_2d,test_maze.rows,test_maze.cols)
start_time = time.time()
val,counter,no_iteration=mdp_value_iteration(test_maze_2d,((2*(test_maze.rows))-1),((2*(test_maze.cols))-1),reward_matrix,discount_factor=0.9, tolerance=0.0001)
exec_time = (time.time() - start_time)
print("--- Running Time : %s seconds ---" % exec_time)
pt=map_optimal_path_maze(val,k)
a=agent(test_maze,footprints=True,color=COLOR.yellow,shape='arrow',filled=True)
print("lengthpath",len(pt))
test_maze.tracePath({a:pt},delay=50)
test_maze.run()


Done
no_iteration 94
Optimal path: [(1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6), (1, 7), (2, 7), (3, 7), (3, 8), (3, 9), (4, 9), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (4, 13), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (4, 21), (5, 21), (6, 21), (7, 21), (7, 22), (7, 23), (8, 23), (9, 23), (9, 24), (9, 25), (9, 26), (9, 27), (10, 27), (11, 27), (12, 27), (13, 27), (14, 27), (15, 27), (15, 28), (15, 29), (15, 30), (15, 31), (15, 32), (15, 33), (16, 33), (17, 33), (18, 33), (19, 33), (20, 33), (21, 33), (21, 34), (21, 35), (22, 35), (23, 35), (23, 36), (23, 37), (24, 37), (25, 37), (25, 36), (25, 35), (26, 35), (27, 35), (28, 35), (29, 35), (30, 35), (31, 35), (32, 35), (33, 35), (34, 35), (35, 35), (35, 34), (35, 33), (36, 33), (37, 33), (37, 34), (37, 35), (38, 35), (39, 35), (39, 36), (39, 37), (39, 38), (39, 39)]
counter 92
--- Running Time : 0.7430729866027832 seconds ---
lengthpath 46
